In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

df = pd.read_csv('/ML/data.csv")

df = df.drop(columns=['date'])

num_days = 10

start_date = datetime.strptime('2024-01-28', '%Y-%m-%d')

def generate_delay():
    rand = np.random.rand()
    if rand < 0.65:
        return np.random.randint(0, 10)  # 0-9 minutes
    elif rand < 0.87:
        return 60  # around 1 hour
    else:
        return np.random.randint(61, 120)  # 61-119 minutes

all_days_data = []

for day in range(num_days):
    current_date = start_date + timedelta(days=day)
    day_str = current_date.strftime('%Y-%m-%d')
    daily_df = df.copy()
    daily_df['date'] = day_str
    # Generate delays
    daily_df['actual_delay_minute'] = daily_df.apply(lambda row: generate_delay(), axis=1)
    # Update cancellation_flag based on delay (e.g., delay > 120 minutes)
    daily_df['cancellation_flag'] = np.where(daily_df['actual_delay_minute'] > 120, 1, 0)
    all_days_data.append(daily_df)

final_df = pd.concat(all_days_data, ignore_index=True)

final_df = final_df[['date', 'train_number', 'from_station', 'to_station',
                     'scheduled_departure', 'scheduled_arrival',
                     'actual_delay_minute', 'cancellation_flag']]

final_df.to_csv('3_days_train_delays.csv', index=False)
print("CSV file '3_days_train_delays.csv' has been generated successfully.")

In [ ]:
dataframe=pd.read_csv('/content/3_days_train_delays.csv')
dataframe.head()
print(dataframe.dtypes)
train_number=int(input("enter the train number"))
src=input("enter the source")
dest=input("enter the dest")
retrieved_data=dataframe[(dataframe["train_number"]==train_number)&(dataframe["from_station"]==src)&(dataframe["to_station"]==dest)]

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
def forecast_time(past_data):
    series = pd.Series(past_data)

    model = ARIMA(series, order=(1, 1, 1))
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=1)
    return forecast
import numpy as np
past_data = np.asarray(retrieved_data["actual_delay_minute"])
ans=forecast_time(past_data)
print(ans)